# Chapter 18: Demo Time Series với PMDARIMA

In [ ]:
# from google.colab import drive
# drive.mount("/content/gdrive", force_remount=True)
# %cd '/content/gdrive/My Drive/LDS6_MachineLearning/practice/Chapter18_ARIMA/'

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv("electric_production.csv",index_col=0)
data.head()

In [ ]:
data.index = pd.to_datetime(data.index)

In [ ]:
data.index

In [ ]:
data.info()

In [ ]:
data.columns = ['Energy Production']

In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
data_1985 = data[data.index.year >=int(1985)]
data_1985.head()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(data_1985)
plt.title("Energy Production Jan 1985--Jan 2018")
plt.show()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(data_1985, model='multiplicative')
result

In [ ]:
result.plot()
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(result.trend)
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(result.seasonal)
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
plt.plot(result.resid)
plt.show()

- Với kết quả trên, ta có thể thấy rõ tính seasonal component của data, và cũng có thể thấy xu hướng dữ liệu ở trên được tách riêng.

- Trend có thể lên hoặc xuống và có thể tuyến tính hoặc phi tuyến tính. Cần phải hiểu tập dữ liệu để biết liệu một khoảng thời gian đáng kể đã trôi qua có thể xác định xu hướng thực tế hay chưa.
- Cũng có thể có biến động bất thường (Irregular fluctuation) là những thay đổi đột ngột ngẫu nhiên và không thể đoán trước

## Áp dụng  auto_arima để xây dựng mô hình
### Cài pip install pmdarima

In [ ]:
# ! pip install pmdarima

In [ ]:
from pmdarima import auto_arima

In [ ]:
stepwise_model = auto_arima(data_1985, start_p=1, start_q=1,
                           max_p=5, max_q=5, m=12,
                           start_P=1, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

In [ ]:
print(stepwise_model.aic())

In [ ]:
train = data.loc['1985-01-01':'2015-01-01'] # 1/1985 => 12/2014
test = data.loc['2015-01-01':] # 1/2015 -> het

In [ ]:
test.head()

In [ ]:
len(test) # so mau trong test

### Bước 2: Fit mô hình

In [ ]:
stepwise_model.fit(train)

### Bước 3: Dự đoán kết quả

In [ ]:
future_forecast = stepwise_model.predict(n_periods=len(test)) # so khoang thoi gian can du doan

In [ ]:
future_forecast

In [ ]:
future_forecast = pd.DataFrame(future_forecast,index = test.index,columns=['Prediction'])
future_forecast

### Bước 4: Trực quan hóa dữ liệu

In [ ]:
plt.plot(test, label='Energy Production')
plt.plot(future_forecast, label='Prediction')
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(data_1985, label='Energy Production')
plt.plot(future_forecast, label='Prediction', color='red')
plt.xticks(rotation='vertical')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
mae = mean_absolute_error(test,future_forecast)
print(mae)


### Dự đoán 12 tháng tiếp theo

In [ ]:
future_forecast = stepwise_model.predict(n_periods=len(test)+12)  # 44 thang + 12 thang can du doan
future_forecast

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(future_forecast[len(test):], color='red')
plt.xticks(rotation='vertical')
plt.title("Prediction next 12 months")
plt.show()

In [ ]:
future_forecast[len(test):]

In [ ]:
months = pd.date_range('2018-09-01','2019-08-01', 
              freq='MS').strftime("%Y-%m-%d").tolist()          

In [ ]:
months

In [ ]:
new_predict = pd.DataFrame({
    'Date' : months,
    'Energy Production': future_forecast[len(test):]}    
    )
new_predict

In [ ]:
# Source: https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c
# Link: https://medium.com/analytics-vidhya/time-series-forecasting-sarima-vs-auto-arima-models-f95e76d71d8f